In [102]:
from IPython.core.display import display, HTML
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
import os
import pandas as pd
from datetime import datetime as dt
import time
import plotly.express as px
import mysql.connector
from mysql.connector import Error
import numpy as np
import config


# defining some widgets
calendar = widgets.DatePicker(
           description='Select Date')

remap = {'13:00 PM':'1:00 PM',
'14:00 PM':'2:00 PM',
'15:00 PM':'3:00 PM',
'16:00 PM':'4:00 PM',
'17:00 PM':'5:00 PM',
'18:00 PM':'6:00 PM',
'19:00 PM':'7:00 PM',
'20:00 PM':'8:00 PM',
'21:00 PM':'9:00 PM',
'22:00 PM':'10:00 PM',
'23:00 PM':'11:00 PM'}

def get_data():
    host = config.host
    database = config.database
    user = config.user
    password = config.password
    conn = mysql.connector.connect(
        host=host, database=database, user=user, password=password)
    appointments = "SELECT \
                    wp_jet_appointments.ID, \
                    wp_jet_appointments.status, \
                    a.post_title as service, \
                    b.post_title as provider, \
                    d.meta_value as dockage, \
                    wp_jet_appointments.user_email, \
                    wp_jet_appointments.date, \
                    wp_jet_appointments.slot AS start_time, \
                    wp_jet_appointments.slot_end AS end_time, \
                    wp_jet_appointments.slot_end - wp_jet_appointments.slot as duration, \
                    wp_jet_appointments.order_id AS confirmation, \
                    u.display_name as captain, \
                    c.meta_value AS boat_length, \
                    wp_jet_appointments.phone, \
                    wp_jet_appointments.comments \
                FROM wp_jet_appointments \
                INNER JOIN wp_posts AS a ON wp_jet_appointments.service = a.ID \
                INNER JOIN wp_posts AS b ON wp_jet_appointments.provider = b.ID \
                INNER JOIN wp_postmeta AS d ON wp_jet_appointments.provider = d.post_id \
                INNER JOIN wp_users AS u ON wp_jet_appointments.user_id = u.ID \
                INNER JOIN wp_usermeta AS c ON u.ID = c.user_id \
                WHERE order_id > 0 \
                AND d.meta_key = 'dock_length' \
                AND c.meta_key = 'length' \
                ORDER BY date, start_time ASC"

    ap_data = pd.read_sql_query(appointments, conn)

    ap_data.to_csv("assets/data/wp_jet_appointments.csv", index=False)
    ap = pd.read_csv("assets/data/wp_jet_appointments.csv")

    # Using a unix epoch time
    ap["date"] = pd.to_datetime(ap["date"], unit='s')
    ap["start_time"] = pd.to_datetime(ap["start_time"], unit='s').dt.time
    ap["end_time"] = pd.to_datetime(ap["end_time"], unit='s').dt.time
    ap["start_time"] = ap["start_time"].apply(lambda x: x.strftime('%I:%M %p'))
    ap["end_time"] = ap["end_time"].apply(lambda x: x.strftime('%I:%M %p'))

    dock_data = ap[["provider", "dockage", "date", "start_time", "end_time", "duration", "boat_length"]]

    times = []
    for k, v in dock_data.iterrows():
        time = pd.date_range(v.start_time, v.end_time, freq="60min",).time
        times.append([v.boat_length, v.provider, v.date, v.dockage, time])

    df = pd.DataFrame(
        times, columns=['boat_length', 'provider', 'date', 'dockage', 'hour'])
    df['hour'] = df.hour.str[:-1]

    df2 = df.explode(column='hour')
    df2['hour'] = df2.hour.apply(lambda x: x.strftime('%H:%M %p'))
    
    if calendar.value is None:
        datestring = dt.now().strftime('%Y-%m-%d')
    else:
        datestring =calendar.value.strftime('%Y-%m-%d')

    df2 = df2[df2.date == calendar.value.strftime('%Y-%m-%d')]
    df3 = pd.DataFrame(df2.groupby(['provider', 'hour'])['boat_length'].sum())
    dockage_key = ap[['provider','dockage']].drop_duplicates().set_index('provider')
    output = df3.join(dockage_key)
    output['availability'] = output.dockage-output.boat_length
    output['date'] = calendar.value.strftime('%Y-%m-%d')
    output = output.reset_index(level=1).replace(remap)
    return display(output[['hour','date','boat_length','dockage','availability']])

# Handle Button
button = widgets.Button(description='Get Data')
out = widgets.Output()
def on_button_clicked(_):
      # "linking function with output"
    with out:
          # what happens when we press the button
        clear_output()
        get_data()
info = Markdown("""<img src="assets/image/DD-Vector-Updated-2.png" align="center"/>""")
display(info)
# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
box = widgets.HBox([calendar,button])
widgets.VBox([box,out])

<img src="assets/image/DD-Vector-Updated-2.png" align="center"/>